In [ ]:
import os
import urllib.request

from astropy.io import fits as f
from astropy.table import Table
from astropy.io import ascii
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook


In [ ]:
name = '{{ NAME }}'
hsla_tarball_url = "https://archive.stsci.edu/missions/hst/spectral_legacy/datapile_05-15-2018_COS/" + name + "/" + name + "_target.tar.gz"
hsla_tarball = name + "_target.tar"
gzipball, headers = urllib.request.urlretrieve(hsla_tarball_url, filename=hsla_gzipped)
!gunzip $hsla_tarball
!tar -xvf $hsla_tarball

## Now you should have some images, tables, and fits files in your pwd


## First, here are a bunch of functions adapted from https://github.com/peeples/Spectator/blob/master/spectator/quick_look.py 

In [ ]:

#-----------------------------------------------------------------------------------------------------

def smooth_spectrum(method, values, weights, scale):
    ## this theoretically allows users to choose if they want binning, Guassian smoothing, or moving average
    ## of course I only have moving average so far soooo.....:
    smooth = movingaverage(values, weights, scale)

    return smooth
      


def movingaverage(values, weights, window):
    indices = np.where(weights > 0)
    mvavg = np.convolve(values[indices], np.ones(window)/window,'same')                    
    return mvavg

#-----------------------------------------------------------------------------------------------------
  

def get_default_windows():
    window = np.array([[1145,1155],
                       [1245,1255],
                       [1345,1355],
                       [1445,1455],
                       [1545,1555],
                       [1645,1655],
                       [1745,1755],
                       [1845,1855],
                       [1920,1930],
                       [2020,2030],
                       [2120,2130],
                       [2220,2230],
                       [2320,2330],
                       [2420,2430],
                       [2520,2530],
                       [2620,2630],
                       [2720,2730],
                       [2820,2830],
                       [2920,2930],
                       [3020,3030],
                       [3120,3130]])
    wc = ['indigo', 'violet', 'blue', 'green', 'gold', 'orange', 'red', 'darkred',
          'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey']

    w_used = np.zeros(21)
    
    return window, wc, w_used


def get_default_pid_colors(pids):
    color_list = ['purple','yellow','red','cyan','orange','green','blue','magenta','tan',
            'grey','pink','maroon','aqua','brown','greenyellow','olive','thistle']
    colors = {}
    for p in np.arange(len(pids)):
        colors[str(pids[p][0])] = color_list[p]
    return colors
    
#-----------------------------------------------------------------------------------------------------




def plot_spectrum(output_name, wavelength, flux, wavemin, wavemax, fluxmin, fluxmax, **kwargs):
    error = kwargs.get("error", None)
    labeltext = kwargs.get("labeltext", "")
    window = kwargs.get("window", get_default_windows()[0])
    wc = kwargs.get("wc", get_default_windows()[1])
    w_used = kwargs.get("w_used", get_default_windows()[2])
    smooth = kwargs.get("smooth", 1)
    wgt = kwargs.get("wgt", np.ones(np.shape(flux)))
    time = kwargs.get("time",-1)
    time_flux = kwargs.get("time_flux", [])

    if (fluxmax < 0): 
        fluxmax = 1.8 * np.mean(flux[np.where(flux > 0.)])
        print("fluxmax received by plot_spectrum is negative: adjusting it to: ", fluxmax)
    
    fig = plt.figure(figsize=(9, 3))
    ax = fig.add_subplot(111)
    if np.asarray(np.shape(np.shape(flux))) > 1:
        for i in range(np.shape(flux)[0]):
            indices = np.where(wgt[i] > 0)
            f = smooth_spectrum(1, flux[i], wgt[i], smooth)
            wave = wavelength[i][indices]
            ax.step(wave, f, lw=1, color='black')
    else:
        indices = np.where(wgt > 0)
        f = smooth_spectrum(1, flux, wgt, smooth)
        wave = wavelength[indices]
        ax.step(wave, f, lw=1, color='black')
    if "error" in kwargs:
        if np.asarray(np.shape(np.shape(flux))) > 1:
            for i in range(np.shape(flux)[0]):
                indices = np.where(wgt[i] > 0)
                e = smooth_spectrum(1, error[i], wgt[i], smooth)
                wave = wavelength[i][indices]
                ax.step(wave, e, lw=1, color='grey', alpha=0.6)
        else:
            e = smooth_spectrum(1, error, wgt, smooth)
            ax.step(wave, e, lw=1, color='grey', alpha=0.6)
        for w in range(np.shape(window)[0]):
            if np.asarray(np.shape(np.shape(flux))) > 1:
                for i in range(np.shape(flux)[0]):
                    sn_all = smooth_spectrum(1, np.ravel(flux[i]/error[i]), wgt[i], smooth)
                    f = smooth_spectrum(1, flux[i], wgt[i], smooth)
                    e = smooth_spectrum(1, error[i], wgt[i], smooth)
                    indices = np.where(wgt[i] > 0)
                    wave = wavelength[i][indices]
                    indices = np.where((f > 0) & (wave > window[w][0]) & (wave < window[w][1]))
                    if(np.shape(indices)[1] > 0):
                        #print "CALCULATING SN FOR ",w, wc[w]
                        medflux = np.median(f[indices])
                        err = np.sqrt(np.average(np.average((medflux-f)**2, weights=e)))/np.sqrt(np.size(indices))
                        sn = np.median(sn_all[indices]) * np.sqrt(smooth)  ## assuming smoothing by one resel
                        if (sn > 0):
                            plt.text(window[w][1], 0.75*fluxmax, "S/N="+"{:.1f}".format(sn), fontsize=10)
                            if w_used[w] == 0:
                                plt.axvspan(window[w][0], window[w][1], facecolor=wc[w], alpha=0.5)
                                w_used[w] = 1
                        print('S to N 1:', time,medflux,err,w, sn)
                        if time > 0 and wc[w] != 'grey':
                            time_flux.append([time,medflux,err,w])
            else:
                f = smooth_spectrum(1, flux, wgt, smooth)
                e = smooth_spectrum(1, error, wgt, smooth)
                sn_all = smooth_spectrum(1, np.ravel(flux/error), wgt, smooth)
                indices = np.where((f > 0) & (wgt > 0) & (wavelength > window[w][0]) & (wavelength < window[w][1]))
            if(np.shape(indices)[1] > 0):
                print("CALCULATING SN FOR ",w, wc[w])
                medflux = np.median(f[indices])
                err = np.sqrt(np.average(np.average((medflux-f)**2, weights=e)))/np.sqrt(np.size(indices))
                sn = np.median(sn_all[indices]) * np.sqrt(smooth)
                print('S to N 2a:', time, medflux, err, w, medflux / err, sn)
                if (sn > 0):
                    plt.text(window[w][1], 0.75*fluxmax, "S/N="+"{:.1f}".format(sn), fontsize=10)
                    if w_used[w] == 0:
                        plt.axvspan(window[w][0], window[w][1], facecolor=wc[w], alpha=0.5)
                        w_used[w] = 1
                print('S to N 2b:', time,medflux,err,w, sn)
                if time > 0 and wc[w] != 'grey':
                    time_flux.append([time,medflux,err,w])
            # plt.axvspan(window[w][0], window[w][1], facecolor=wc[w], alpha=0.5)

    # do we want to overplot any other spectra?
    if "overwave" in kwargs and "overflux" in kwargs:
        print("overplotting something")
        overwavelength = kwargs.get("overwave", [])
        overflux = kwargs.get("overflux", [])
        overerror = kwargs.get("overerror", [])
        overwgt = kwargs.get("overwgt", np.ones(np.shape(overflux)))
        overcolor = kwargs.get("overcolor", "blue")
        if np.asarray(np.shape(np.shape(overflux))) > 1:
            for i in range(np.shape(overflux)[0]):
                indices = np.where(overwgt[i] > 0)
                f = smooth_spectrum(1, overflux[i], overwgt[i], smooth)
                overwave = overwavelength[i][indices]
                print("plot here", i)
                ax.step(overwave, f, lw=1, color=overcolor, zorder=1)
            if "overerror" in kwargs:
                for i in range(np.shape(overflux)[0]):
                    e = smooth_spectrum(1, overerror[i], overwgt[i], smooth)
                    ax.step(overwave, e, lw=1, color=overcolor, zorder=10, alpha=0.2)
        else:
            print("or plot here")
            indices = np.where(overwgt > 0)
            f = smooth_spectrum(1, overflux, overwgt, smooth)
            overwave = overwavelength[indices]
            ax.step(overwave, f, lw=1, color=overcolor, zorder=2)
            if "overerror" in kwargs:
                e = smooth_spectrum(1, overerror, overwgt, smooth)
                ax.step(overwave, e, lw=1, color=overcolor, zorder=10, alpha=0.2)

    plt.xlim(wavemin, wavemax)
    plt.ylim(fluxmin, fluxmax)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.ylabel(r'flux [erg / s / cm$^2/\AA$]', fontsize=14)
    plt.xlabel(r'wavelength [$\AA$]',fontsize=14)
    plt.tight_layout()
    plt.text(0.99, 0.88, labeltext, fontsize=11, transform=ax.transAxes, horizontalalignment='right', color='black', bbox=dict(facecolor='white',linewidth=0.3,alpha=0.5), zorder=15)
    
    ## if os.path.isfile(output_name):   os.system("rm -f " + output_name) 
    plt.show()
    ## plt.savefig(output_name)
    ## plt.close(fig)

    return time_flux




## OK, now to see if there is a FUVM co-added spectrum, read it in, and plot it

In [ ]:
## Is there an FUVM co-add?
window, wc, w_used = get_default_windows()
fuvm_file = name + '_coadd_FUVM_final_lpALL.fits.gz'
print('checking for ',fuvm_file)
if(os.path.exists(fuvm_file)):
    coadd_exists = True
    output_name = name + '_coadd_final_all.png'
    labeltext = """full coadd of """+name+""" COS/FUV M"""
    print('      ~~~ happy fuv data dance ~~~~')
    print('      YES!! I FOUND THE FULL G130M+G160M COADD!')
    print('      ~~~~ happy fuv data dance ~~~')
    coadd = Table.read(fuvm_file, format='fits')
    MAX_FLUX = 1.05*np.max(coadd['FLUX'])
    plot_spectrum(output_name, coadd['WAVE'], coadd['FLUX'], 1100, 1900, 0, MAX_FLUX, window=window, wc=wc, labeltext=labeltext, error=coadd['ERROR'], smooth=6)
